In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/19

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import transforms, datasets, models
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18, ResNet18_Weights


import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()


Training on cpu
Flower 1.13.1 / PyTorch 2.5.1+cu121


This the same dataset. I will be tweaking some settings in order to get better accuracy. Currently I am getting 43% I aim for 90%+

I will be using a pretrained resnet model. I have edited the final layer so its 10 labels.

In [ ]:
class CIFAR10DictWrapper(Dataset):
    """
    A wrapper around the CIFAR-10 dataset to convert output to dictionary format.
    """
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return {"img": img, "label": label}

def load_cifar10_centralized(batch_size=128):
    """
    Load CIFAR-10 dataset in centralized mode and return DataLoaders with dictionary key format.

    Args:
        batch_size (int): Batch size for DataLoader.

    Returns:
        Tuple[DataLoader, DataLoader]: Train and test DataLoaders.
    """
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ])

    # Load CIFAR-10 datasets
    trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    # Wrap the datasets to return dictionary format
    trainset = CIFAR10DictWrapper(trainset)
    testset = CIFAR10DictWrapper(testset)

    # Create DataLoaders
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader


In [ ]:
NUM_CLIENTS=10
BATCH_SIZE=64

def load_datasets(partition_id: int):
    """
    Load datasets for both centralized and federated learning.

    Args:
        partition_id (int):
            For federated learning, this specifies the client partition ID.
            For centralized learning, set partition_id=0 to use the full dataset.

    Returns:
        Tuple[DataLoader, DataLoader, DataLoader]: Train, validation, and test loaders.
    """
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    # Divide data into 80% train, 20% validation for each partition
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)

    pytorch_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])

    def apply_transforms(batch):
        # Apply transforms to the "img" key in the batch
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    # Apply transforms and create DataLoaders
    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    return trainloader, valloader, testloader


In [ ]:
class ResNetCIFAR10(nn.Module):
    def __init__(self):
        super(ResNetCIFAR10, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 10)

    def forward(self, x):
        return self.resnet(x)


We initialize the cnn as it works great with images

We initialize the parent class of Net which is nn.Module.  
Then we start registering our layers. First layer in a CNN is often a convolutional layer. This layer usually has 3 inputs since there are 3 rgb channels for images. This layer applies a filter to find the edges, textures or objects in the image. We also define the output as 6 feature maps. And finally we apply a 5x5 filter to do the filtering.

We then have a pooling layer which is a maximum pooling layer. This layer essentially takes the max value from a 2x2 region meaning, keeps the important features and discards the rest while retaining feature map count.

We then have another conv layer does the same thing but now there are 16 feature maps. Helps extract more complex features from the pooling layer.

We then have our first fully connected layer. Our input is the flattened features from the convolutional layers. 16 feature maps then 5x5 for the size of the features. We then output 120 neurons

Second FC layer. takes 120 inputs gives 84 outputs. Processes higher level features and "compresses" the model making it more dense.

Final layer takes those 84 features as input and produces 10 outputs which are the labels from the CIFAR-10 dataset.


We then have the forward pass defined. x is passed in which is the image data.
We then apply the mentioned layers so conv layer then relu actv func is applied. Then we apply the pool layer. This reduces the computational load making the model more efficent.  

The same process is than repeated with the conv2 layer.
We than flatten to a 1D vector to prepare for the fully connected layer.

Then we pass the image data thru the first fully connected layer andthen use ReLu activation. Repeat for the next step just making it more denser
We than have the last layer where we get the 10 labels.


In [ ]:

def train(net, trainloader, epochs, lr=0.1):
    """Train the network."""
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    optimizer = torch.optim.Adam(net.parameters())

    net.train()


    for epoch in range(epochs):
        running_loss, correct, total = 0.0, 0, 0
        for batch in trainloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)  # Access dictionary keys
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        #scheduler.step()
        print(f"Epoch {epoch+1}/{epochs}: Loss={running_loss/len(trainloader):.4f}, Accuracy={100*correct/total:.2f}%")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy


The train function defines the optimizer and the loss function to be used in the training program

Then we put the model in to training mode in line 6

Then we run the epochs for the specified amount.
We load the batches and move them to the specified device in this case gpu

We zero the gradient to avoid accumulation.

We perform a forward pass line 12 gathers the predicted outputs,then next line we calculate the loss giving it the predicted and the true labels.

Then we do a backward pass and run the optimizer which is adam in this case.

We gather out loss for accuracy calculation then for verbose we have the verbose print statement.

for he test function we are using the same loss func. then we set the model in eval mode.

disable gradient tracking to save memory and speed up computation

we itirate over the batches in tetloader. then we do a foward pass and compute the loss.



BELOW CELL

we load the data then crate a CNN instance.

we than have the training loop for 5 epochs
we evaluate the model using test defined above

and we do final test to evalute the model once the epochs are over.


In [ ]:
trainloader, testloader = load_cifar10_centralized()
net = ResNetCIFAR10().to(DEVICE)
train(net, trainloader, epochs=20, lr=0.1)
test(net, testloader)


100%|██████████| 170M/170M [00:03<00:00, 47.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 97.7MB/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Set and get paramters used by the FL algo to get weights and biases.

In [ ]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

We implement the client here extending the flwr.client.NumPyClient

we init the client with local instances of the neural network,
the train data and validation data set are also loaded. In a real environment client would load its own dataset and its own model instance.

get parameters send the clients current model parameters to the server. This funtion is used by the server to get the model paramters for aggregation.

the fit function received the global params and trains the model locally on client dataset.

the evaluate function evaluates the model using the validation dataset and sends the results to the server

In [ ]:
class FederatedClient(flwr.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.net.state_dict().values()]

    def fit(self, parameters, config):
      self.set_parameters(parameters)
      lr = config["lr"]
      local_epochs = config["local_epochs"]
      train(self.net, self.trainloader, epochs=local_epochs, lr=lr)
      return self.get_parameters(config), len(self.trainloader.dataset), {}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader.dataset), {"accuracy": float(accuracy)}

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)


This is the function to crate the client.
We specify device type cpu or gpu
load the data
then return the client instance
client app is used to managed the clients

In [ ]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = ResNetCIFAR10().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    trainloader, valloader, testloader=load_datasets(partition_id=partition_id)

    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return FederatedClient(net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

#For creating clients.

We define the strat that we are going to be using here

In [ ]:
# Create FedAvg strategy
strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
    on_fit_config_fn=lambda round: {"lr": 0.1, "local_epochs": 1},  # Pass to clients
)

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """


    config = ServerConfig(num_rounds=20)

    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

we define our server on the code above. Set the number of rounds we want to do.
then we just use serverappcomponents to initilize the server

BELOW CELL
we give the resources we have depending on device type

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

Here we run the simulation using the defined server and client app, number of clients and our config.

In [ ]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=10,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=20, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=49144) 2024-12-16 18:26:10.296706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=49144) 2024-12-16 18:26:10.338256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=49144) 2024-12-16 18:26:10.348979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=49144) 2024-12-16 18:26:12.081306: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(ClientAppActor p

(ClientAppActor pid=49144) Epoch 1/1: Loss=6.5968, Accuracy=11.00%


(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=49144)   warnings.warn(
(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(ClientAppActor pid=49144)   warnings.warn(msg)


(ClientAppActor pid=49144) Epoch 1/1: Loss=4.7521, Accuracy=12.97%


(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=49144)   warnings.warn(
(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(ClientAppActor pid=49144)   warnings.warn(msg)


...

(ClientAppActor pid=49144) Epoch 1/1: Loss=4.0726, Accuracy=10.60%


(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=49144)   warnings.warn(
(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(ClientAppActor pid=49144)   warnings.warn(msg)


(ClientAppActor pid=49144) Epoch 1/1: Loss=4.0501, Accuracy=11.72%


(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=49144)   warnings.warn(
(ClientAppActor pid=49144) /usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(ClientAppActor pid=49144)   warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

same process as before just this time we add the weigted avarage function to see our accuracy.

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=10,
        min_evaluate_clients=5,
        min_available_clients=10,
        on_fit_config_fn=lambda round: {"lr": 0.1, "local_epochs": 5},  # Pass to clients
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=10)

    return ServerAppComponents(strategy=strategy, config=config)


# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=10,
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=4471) 2024-12-16 20:18:33.326830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=4471) 2024-12-16 20:18:33.373302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=4471) 2024-12-16 20:18:33.391623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=4471) 2024-12-16 20:18:36.204951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning

(ClientAppActor pid=4471) Epoch 1/5: Loss=1.7533, Accuracy=37.88%
(ClientAppActor pid=4470) Epoch 1/5: Loss=1.8068, Accuracy=36.05%
(ClientAppActor pid=4471) Epoch 2/5: Loss=1.3144, Accuracy=55.40%
(ClientAppActor pid=4470) Epoch 2/5: Loss=1.3286, Accuracy=53.27%
(ClientAppActor pid=4470) Epoch 3/5: Loss=1.1312, Accuracy=60.40%
(ClientAppActor pid=4470) Epoch 4/5: Loss=1.0079, Accuracy=65.83% [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(ClientAppActor pid=4471) Epoch 4/5: Loss=1.0564, Accuracy=64.05%
(ClientAppActor pid=4470) Epoch 5/5: Loss=0.9145, Accuracy=69.17%
(ClientAppActor pid=4471) Epoch 5/5: Loss=0.9838, Accuracy=66.83%
(ClientAppActor pid=4470) Epoch 1/5: Loss=1.7061, Accuracy=40.02%
(ClientAppActor pid=4471) Epoch 1/5: Loss=1.7494, Accuracy=38.73%
(ClientAppActor pid=4470) Epoch 2/5: Loss=1.2915, 

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=4470) Epoch 1/5: Loss=1.1728, Accuracy=60.02%
(ClientAppActor pid=4470) Epoch 2/5: Loss=1.0234, Accuracy=66.58% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 2/5: Loss=1.0308, Accuracy=66.12%
(ClientAppActor pid=4470) Epoch 3/5: Loss=0.9249, Accuracy=69.92%
(ClientAppActor pid=4471) Epoch 3/5: Loss=0.9023, Accuracy=69.75%
(ClientAppActor pid=4470) Epoch 4/5: Loss=0.8611, Accuracy=71.22%
(ClientAppActor pid=4471) Epoch 4/5: Loss=0.8893, Accuracy=70.33%
(ClientAppActor pid=4470) Epoch 5/5: Loss=0.7929, Accuracy=73.67%
(ClientAppActor pid=4471) Epoch 5/5: Loss=0.8226, Accuracy=72.45%
(ClientAppActor pid=4470) Epoch 1/5: Loss=1.1693, Accuracy=61.12%
(ClientAppActor pid=4471) Epoch 1/5: Loss=1.1758, Accuracy=60.58%
(ClientAppActor pid=4470) Epoch 2/5: Loss=1.0229, Accuracy=66.67%
(ClientAppActor pid=4471) Epoch 2/5: Loss=1.0256, Accuracy=65.53%
(ClientAppActor pid=4470) Epoch 3/5: Loss=0.9769, Accuracy=67.50%
(ClientAppActor pid=4471) Epoch 3/5: Loss=0.945

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=4470) Epoch 1/5: Loss=0.9769, Accuracy=68.33%
(ClientAppActor pid=4471) Epoch 2/5: Loss=0.8411, Accuracy=72.05% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 3/5: Loss=0.7868, Accuracy=74.45% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 4/5: Loss=0.7414, Accuracy=75.40% [repeated 2x across cluster]
(ClientAppActor pid=4470) Epoch 5/5: Loss=0.6779, Accuracy=76.90% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 1/5: Loss=0.9536, Accuracy=68.72% [repeated 2x across cluster]
(ClientAppActor pid=4470) Epoch 2/5: Loss=0.8317, Accuracy=71.92% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 3/5: Loss=0.7746, Accuracy=73.70% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 4/5: Loss=0.7358, Accuracy=75.20% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 5/5: Loss=0.7059, Accuracy=76.20% [repeated 2x across cluster]
(ClientAppActor pid=4471) Epoch 1/5: Loss=0.9029, Accuracy=69.40% [repeated 2x 